<a href="https://colab.research.google.com/github/bhushanmandava/LLM_fine_tuning/blob/main/Movie_sentiment_Analysis(Bert).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine tuning LLm for  Movie Sentiment Analysis

In [5]:
!pip install datasets
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset,DatasetDict,Dataset
from transformers import (
AutoTokenizer,
AutoConfig,
AutoModelForSequenceClassification,
DataCollatorWithPadding,
TrainingArguments,
Trainer
)

from peft import PeftModel , PeftConfig,get_peft_model,LoraConfig
import evaluate
import torch
import numpy as np

In [7]:
dataset = load_dataset('shawhin/imdb-truncated')
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

(…)-00000-of-00001-5a744bf76a1d84b2.parquet:   0%|          | 0.00/836k [00:00<?, ?B/s]

(…)-00000-of-00001-a3a52fabb70c739f.parquet:   0%|          | 0.00/853k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [8]:
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])
#so we can say labels are equally distributed and we are giving enough chances for model to learn positive and negative sentiment

np.float64(0.5)

In [34]:
#starting with or base model
model_checkpoint = 'distilbert-base-uncased'


#defining our label maps
id2label = {
    0:'Negative',
    1:'Positive'
}
label2id = {"Negative":0, "Positive":1}
# using AutoModelForSeuentialclassification  changing our model heads
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
model


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU if available
model.to(device)
print(f"Using device: {device}")


Using device: cuda


You can seee clearly that it only uses The Encoder part of the transformer

## Preprocessing The data

In [37]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)# getting our model specific tokenizer
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token':'<pad>'})#if the tokinzer dosent have pad ytoken we adding our own
    model.resize_token_embeddings(len(tokenizer))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [38]:
def tokinizer_function(examples):
    text= examples["text"]
    tokenizer.truncation_side ="left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [39]:
#mapping our function
tokenized_dataset = dataset.map(tokinizer_function,batched = True)
tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

### now our data is preprocessed bassed on our model requirements

In [40]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [41]:
accuracy = evaluate.load("accuracy")
#our evaluation metric would be accuracy

In [42]:
def compute_metrics(p):
    predictions, labels = p
    # If it's a binary classification task
    if predictions.ndim == 1 or predictions.shape[1] == 1:
        pred = np.round(predictions)
    else:
        # Multi-class classification
        pred = np.argmax(predictions, axis=1)

    accuracy = (pred == labels).mean()
    return {"accuracy": accuracy}


In [43]:
text_list =["it was a good movie","good god,thats the worst film","better than first film","not worth the time"]
print("untrained predictions")
print("*"*50)
for text in text_list:
    inputs=tokenizer.encode(text,return_tensors="pt")
    inputs = inputs.to(device)

    logits = model(inputs).logits
    predictions =torch.argmax(logits)
    print(text + " - " + id2label[predictions.tolist()])


untrained predictions
**************************************************
it was a good movie - Positive
good god,thats the worst film - Positive
better than first film - Positive
not worth the time - Positive


#### see our model is way off so we train our model

In [44]:
for name, module in model.named_modules():
    print(name)



distilbert
distilbert.embeddings
distilbert.embeddings.word_embeddings
distilbert.embeddings.position_embeddings
distilbert.embeddings.LayerNorm
distilbert.embeddings.dropout
distilbert.transformer
distilbert.transformer.layer
distilbert.transformer.layer.0
distilbert.transformer.layer.0.attention
distilbert.transformer.layer.0.attention.dropout
distilbert.transformer.layer.0.attention.q_lin
distilbert.transformer.layer.0.attention.k_lin
distilbert.transformer.layer.0.attention.v_lin
distilbert.transformer.layer.0.attention.out_lin
distilbert.transformer.layer.0.sa_layer_norm
distilbert.transformer.layer.0.ffn
distilbert.transformer.layer.0.ffn.dropout
distilbert.transformer.layer.0.ffn.lin1
distilbert.transformer.layer.0.ffn.lin2
distilbert.transformer.layer.0.ffn.activation
distilbert.transformer.layer.0.output_layer_norm
distilbert.transformer.layer.1
distilbert.transformer.layer.1.attention
distilbert.transformer.layer.1.attention.dropout
distilbert.transformer.layer.1.attention.q

In [46]:
peft_config = LoraConfig(
    task_type ="SEQ_CLS",
    r=4,
    lora_alpha =32,
    lora_dropout=0.1,
target_modules=['q_lin'])

In [47]:
peft_config

LoraConfig(task_type='SEQ_CLS', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, inference_mode=False, r=4, target_modules={'q_lin'}, exclude_modules=None, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, eva_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)

In [48]:
model = get_peft_model(model, peft_config)

In [49]:
# Check the number of trainable parameters in the model
def print_trainable_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"Total parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {total_params - trainable_params}")

# Call the function for your model
print_trainable_parameters(model)


Total parameters: 67584004
Trainable parameters: 628994
Non-trainable parameters: 66955010


In [50]:
# hyperparameters
lr = 1e-5
batch_size = 2
num_epochs = 10

In [51]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [52]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

<ipython-input-52-e8e4c1055773>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.684100,0.670383,0.663000
2,0.660500,0.631024,0.837000
3,0.606600,0.541647,0.868000
4,0.493500,0.395706,0.879000
5,0.376700,0.310003,0.878000
6,0.329000,0.287510,0.878000
7,0.309600,0.290163,0.876000
8,0.313900,0.294083,0.879000
9,0.329100,0.300639,0.876000
10,0.324400,0.303185,0.875000


TrainOutput(global_step=5000, training_loss=0.4427349304199219, metrics={'train_runtime': 393.8293, 'train_samples_per_second': 25.392, 'train_steps_per_second': 12.696, 'total_flos': 920634052343472.0, 'train_loss': 0.4427349304199219, 'epoch': 10.0})

In [53]:
model.to(device)
print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to(device)
    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
it was a good movie - Positive
good god,thats the worst film - Negative
better than first film - Positive
not worth the time - Negative
